In [18]:
from datetime import datetime
import json
import logging
from pathlib import Path
import re
import time
from typing import List, Tuple
from tempfile import NamedTemporaryFile

import dask_geopandas as dgd 
import hydra
import geopandas as gpd
import numpy as np
from omegaconf import DictConfig, OmegaConf
import pandas as pd
from pyproj import CRS
from tqdm.notebook import tqdm
import xarray as xr
import zarr

log = logging.getLogger(__name__)

In [87]:
json_data = '''
{
  "name": "MERIT",
  "data_path": "/data/tkb5476/projects/marquette/data",
  "dx": 2000,
  "buffer": 0.3334,
  "continent": 7,
  "area": 8,
  "chunk_size": 1000,
  "save_name": "${name}_${continent}${area}",
  "save_paths": {
    "huc10": "${data_path}/HUC/huc_10_CONUS.shp",
    "singular_huc10": "${data_path}/HUC/huc_10_separate",
    "flow_lines": "${data_path}/${name}/raw/flowlines",
    "basins": "${data_path}/${name}/raw/basins/cat_pfaf_${continent}${area}_MERIT_Hydro_v07_Basins_v01_bugfix1.shp"
  },
  "zarr": {
    "edges": "${data_path}/${name}/zarr/dpl_v2/${save_name}_edges/",
    "sorted_edges_keys": "${data_path}/${name}/zarr/dpl_v2/${save_name}_edge_keys/",
    "TM": "${data_path}/${name}/zarr/TMs/PFAF_${continent}${area}"
  },
  "csv": {
    "TM": "${data_path}/${name}/processed_csvs/TMs/${continent}${area}_TM.csv.gzip"
  }
}'''

data_dict = json.loads(json_data)
cfg = OmegaConf.create(data_dict)

### Separate HUCs into smaller pieces (ONLY RUN ONCE) 

In [5]:
# gdf = gpd.read_file(Path(cfg.save_paths.huc10))
# grouped = gdf.groupby('HUC10')

In [10]:
# for huc10_value, group in tqdm(grouped, desc='Processing shp files'):
#     out_path = f'{cfg.save_paths.singular_huc10}/{huc10_value}.shp'
#     group.to_file(out_path)

Processing shp files:   0%|          | 0/16164 [00:00<?, ?it/s]

### Create the TM

#### TODO: There may be some MERIT basins that are in many HUC10s. If the routing code has problems, come back to this as it's a complicated fix since the data is so large. 

In [15]:
# centroid_gdf = gpd.GeoDataFrame(joined_gdf[['centroid']], geometry='centroid')
# centroid_gdf.to_file("/data/tkb5476/projects/marquette/data/MERIT/centroid.shp")
# joined_gdf = joined_gdf.drop(columns=['centroid'])
# joined_gdf.to_file("/data/tkb5476/projects/marquette/data/MERIT/joined_MERIT.shp")

/tmp/ipykernel_2956147/3243512046.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  joined_gdf.to_file("/data/tkb5476/projects/marquette/data/MERIT/joined_MERIT.shp")


In [75]:
def join_geospatial_data(cfg: DictConfig) -> gpd.GeoDataFrame:
    """
    Joins two geospatial datasets based on the intersection of centroids of one dataset with the geometries of the other.

    Args:
    huc10_path (str): File path to the HUC10 shapefile.
    basins_path (str): File path to the basins shapefile.

    Returns:
    gpd.GeoDataFrame: The resulting joined GeoDataFrame.
    """
    huc10_gdf = gpd.read_file(Path(cfg.save_paths.huc10)).to_crs(epsg=4326)
    basins_gdf = gpd.read_file(Path(cfg.save_paths.basins))
    basins_gdf['centroid'] = basins_gdf.geometry.centroid
    joined_gdf = gpd.sjoin(basins_gdf.set_geometry('centroid'), huc10_gdf, how='left', op='intersects')
    joined_gdf.set_geometry('geometry', inplace=True)
    return joined_gdf


In [110]:
def create_TM(cfg: DictConfig, gdf: gpd.GeoDataFrame) -> None:
    """
    Create a Transfer Matrix (TM) from GeoDataFrame.

    Args:
        cfg (DictConfig): Hydra configuration object containing settings.
        gdf (GeoDataFrame): GeoDataFrame containing geographical data.
    """
    gdf = gdf.dropna(subset=['HUC10'])
    huc10_ids = gdf["HUC10"].unique()
    merit_ids = gdf["COMID"].unique()
    huc10_ids.sort()
    merit_ids.sort()
    data_array = xr.DataArray(np.zeros((len(huc10_ids), len(merit_ids))),
                              dims=["HUC10", "COMID"],
                              coords={"HUC10": huc10_ids, "COMID": merit_ids})
    for idx, huc_id in enumerate(tqdm(huc10_ids, desc="creating TM")):
        merit_basins = gdf[gdf['HUC10'] == str(huc_id)]
        total_area = merit_basins.iloc[0]["area_new"]

        for j, basin in merit_basins.iterrows():
            unit_area = basin.unitarea / total_area
            data_array.loc[huc_id, basin.COMID] = unit_area
    xr_dataset = xr.Dataset(
        data_vars={"TM": data_array},
        coords={"HUC10": huc10_ids, "COMID": merit_ids},
        attrs={"description": "HUC10 -> MERIT Transition Matrix"}
    )
    print("Saving Zarr Data")
    zarr_path = Path(cfg.zarr.TM)
    xr_dataset.to_zarr(zarr_path, mode='w')
    zarr_hierarchy = zarr.open_group(Path(cfg.zarr.TM), mode='r')
    log.info(f"TM saved to Zarr file at {zarr_path}")
    # print("Saving CSV Data")
    # df = xr_dataset.to_dataframe().unstack('COMID')['TM']
    # df.to_csv(Path(cfg.csv.TM), compression="gzip")
    # log.info("Finished Data Extraction")
    return xr_dataset

# np.random.seed(0)
# temperature = 15 + 8 * np.random.randn(2, 2, 3)
# precipitation = 10 * np.random.rand(2, 2, 3)
# lon = [[-99.83, -99.32], [-99.79, -99.23]]
# lat = [[42.25, 42.21], [42.63, 42.59]]
# time = pd.date_range("2014-09-06", periods=3)
# reference_time = pd.Timestamp("2014-09-05")
# ds = xr.Dataset(
#     data_vars=dict(
#         temperature=(["x", "y", "time"], temperature),
#         precipitation=(["x", "y", "time"], precipitation),
#     ),
#     coords=dict(
#         lon=(["x", "y"], lon),
#         lat=(["x", "y"], lat),
#         time=time,
#         reference_time=reference_time,
#     ),
#     attrs=dict(description="Weather related data."),
# )
# <xarray.Dataset>
# Dimensions:         (x: 2, y: 2, time: 3)
# Coordinates:
#     lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
#     lat             (x, y) float64 42.25 42.21 42.63 42.59
#   * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
#     reference_time  datetime64[ns] 2014-09-05
# Dimensions without coordinates: x, y
# Data variables:
#     temperature     (x, y, time) float64 29.11 18.2 22.83 ... 18.28 16.15 26.63
#     precipitation   (x, y, time) float64 5.68 9.256 0.7104 ... 7.992 4.615 7.805
# Attributes:
#     description:  Weather related data.

In [82]:
def plot_histogram(df: pd.DataFrame, num_bins: int = 100) -> None:
    """
    Creates and displays a histogram for the sum of values in each row of the provided DataFrame.

    Args:
    df (pd.DataFrame): A Pandas DataFrame whose row sums will be used for the histogram.
    num_bins (int, optional): The number of bins for the histogram. Defaults to 100.

    The function calculates the minimum, median, mean, and maximum values of the row sums
    and displays these as vertical lines on the histogram.
    """
    series = df.sum(axis=1)
    plt.figure(figsize=(10, 6))
    series.hist(bins=num_bins)
    plt.xlabel(r'Ratio of  $\sum$ MERIT basin area to HUC10 basin areas')
    plt.ylabel('Number of HUC10s')
    plt.title(r'Distribution of $\sum$ MERIT area / HUC10 basin area')
    min_val = series.min()
    median_val = series.median()
    mean_val = series.mean()
    max_val = series.max()
    plt.axvline(min_val, color='grey', linestyle='dashed', linewidth=2, label=f'Min: {min_val:.3f}')
    plt.axvline(median_val, color='blue', linestyle='dashed', linewidth=2, label=f'Median: {median_val:.3f}')
    plt.axvline(mean_val, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {mean_val:.3f}')
    plt.axvline(max_val, color='green', linestyle='dashed', linewidth=2, label=f'Max: {max_val:.3f}')
    plt.legend()
    plt.show()

In [111]:
# start = time.perf_counter()
# overlayed_merit_basins = join_geospatial_data(cfg)
xr_dataset = create_TM(cfg, overlayed_merit_basins)
# zarr_dataset, df = create_TM(cfg, overlayed_merit_basins)
# plot_histogram(df)
# end = time.perf_counter()
# print(f"This took: {(end - start):.6f} seconds")

creating TM:   0%|          | 0/1573 [00:00<?, ?it/s]

Saving Zarr Data


In [113]:
xr_dataset["HUC10"]

<xarray.DataArray 'HUC10' (HUC10: 1573)>
array(['0904000300', '1701010101', '1701010102', ..., '1711002111',
       '1712000103', '1712000515'], dtype=object)
Coordinates:
  * HUC10    (HUC10) object '0904000300' '1701010101' ... '1712000515'